In [1]:
#importação das dependências necessárias
from numba import vectorize, cuda
import numpy as np
from timeit import default_timer as timer
from math import floor

In [4]:
# funcoes normal para comparacao de tempo
def numerosPrimosCPU(n):
    primos = []
    for i in range(1, n):
        if i % 2 != 0:
            div = 0
            for j in range(3, floor(i/2), 2):
                if i % j == 0:
                    div += 1
                    break
            if div == 0:
                primos.append(i)
 
 
def fatoresNumeroCPU(n):
    fatores = []
    for i in range(1, floor(n/2)):
        if n % i == 0:
            fatores.append(i)
 
 
# funcao paralelizada
# o vectorize faz com que ela seja iterada numa array
@vectorize(["int32(int32)"], target = GPU)  # retorna um int32 e recebe um int 32
def numerosPrimosGPU(a):
    if(a % 2 == 0):
        return 0
    for i in range(3, int(a/2), 2):
        if a % i == 0:
            return 0
    return 1  # retorna 1 se for primo e 0 se nao
 
 
@vectorize(["int32(int32,int32)"])
def fatoresNumeroGPU(n, div):
    if n % div == 0:
        return 1  # retorna 1 se for fator
    return 0  # retorna 0 se nao

NameError: ignored

In [3]:
if __name__ == "__main__":
    numeroLimite = 1000
    testeFatores = 10000000
 
    start = timer()
    fatoresNumeroCPU(testeFatores)
    print(
        f"Tempo para calcular fatores de {testeFatores} sem GPU: {timer()-start}")
 
    start = timer()
    possiveisFatores = np.arange(1, floor(testeFatores/2)+1, dtype=np.int32)
 
    # será uma array com valor booleano de quais numeros sao fatores
    fatores = np.zeros(floor(testeFatores/2)+1, dtype=np.int32)
    fatores = fatoresNumeroGPU(testeFatores, possiveisFatores)
    print(f"Tempo para calcular fatores de {testeFatores} com GPU: {timer()-start}\n")
 
    start = timer()
    numerosPrimosCPU(numeroLimite)
    print(f"Tempo para primos até {numeroLimite} sem GPU: {timer()-start}")
 
    start = timer()
    numerosTestePrimo = np.arange(1, numeroLimite, 2, dtype=np.int32)
 
    # será uma array com valor booleano de quais numeros sao primos
    primos = np.zeros(numeroLimite, dtype=np.int32)
    primos = numerosPrimosGPU(numerosTestePrimo)
    print(f"Tempo para primos até {numeroLimite} com GPU: {timer()-start}")


Tempo para calcular fatores de 10000000 sem GPU: 0.45031077899966476
Tempo para calcular fatores de 10000000 com GPU: 0.05167808300029719

Tempo para primos até 1000 sem GPU: 0.001594623000073625
Tempo para primos até 1000 com GPU: 0.0001496009999755188
